<a href="https://colab.research.google.com/github/fakabbir/AI-Playbook/blob/master/Custom%20NER%20Training%20using%20SpaCy%20%2B%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pip install -U spacy[cuda113]
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 2.1 MB 13.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install cupy-cuda113
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [ ]:
from pprint import pprint
import spacy
print("Spacy version: ",spacy.__version__)
nlp = spacy.load('en_core_web_trf')
pprint("Pipelines: \n")
pprint(nlp.pipeline)

Spacy version:  3.3.0
'Pipelines: \n'
[('transformer',
  <spacy_transformers.pipeline_component.Transformer object at 0x7f4a0e5fd670>),
 ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7f4a0e5d2520>),
 ('parser',
  <spacy.pipeline.dep_parser.DependencyParser object at 0x7f4a0e5d69d0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler object at 0x7f4a0e53adc0>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7f4a0e541780>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7f4a0e5d6cd0>)]


In [ ]:
doc = nlp("Bezo founded Amazon pvt ltd in 2009")

In [ ]:
doc._.trf_data.tensors[0].shape

(1, 12, 768)

In [ ]:
doc._.trf_data.tensors[1].shape

(1, 768)

In [ ]:
doc._.trf_data.model_output.last_hidden_state.shape

(1, 12, 768)

In [ ]:
doc._.trf_data.model_output.pooler_output.shape

(1, 768)

In [ ]:
#doc._.trf_data.model_output.pooler_output == doc._.trf_data.tensors[1] # Gives True

In [ ]:
TRAIN_DATA = [
    ("text1", {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0}})
]

In [ ]:
! curl https://raw.githubusercontent.com/UBIAI/Fine_tune_BERT_with_spacy3/main/test.tsv > test.tsv
! curl https://raw.githubusercontent.com/UBIAI/Fine_tune_BERT_with_spacy3/main/train.tsv > train.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22189  100 22189    0     0  91690      0 --:--:-- --:--:-- --:--:-- 91690
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  151k  100  151k    0     0   856k      0 --:--:-- --:--:-- --:--:--  856k


In [ ]:
for ent in doc.ents:
  print(ent, ent.label_,ent.has_vector)

Bezo PERSON False
Amazon pvt ltd ORG False
2009 DATE False


In [ ]:
!python -m spacy convert train.tsv ./ -t json -n 1 -c iob
!python -m spacy convert test.tsv ./ -t json -n 1 -c iob

ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): train.json
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): test.json


In [ ]:
ls

config.cfg    test.json   test.tsv    train.spacy
sample_data/  test.spacy  train.json  train.tsv


In [ ]:
!python -m spacy convert train.json ./ -t spacy
!python -m spacy convert test.json ./ -t spacy

✔ Generated output file (77 documents): train.spacy
✔ Generated output file (11 documents): test.spacy


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./test.spacy --output ./

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-02 07:17:03,884] [INFO] Set up nlp object from config
[2022-05-02 07:17:03,898] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 07:17:03,902] [INFO] Created vocabulary
[2022-05-02 07:17:05,837] [INFO] Added vectors: en_core_web_lg
[2022-05-02 07:17:07,124] [INFO] Finished initializing nlp object
[2022-05-02 07:17:10,052] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    103.78    2.35    1.74    3.63    0.02
  2     200        280.51   6036.32   62.62   58.90   66.84    0.63
  5     400         47.11   2598.50   62.10   67.48  

In [31]:
ls

config.cfg   model-last/   test.json   test.tsv    train.spacy
model-best/  sample_data/  test.spacy  train.json  train.tsv


In [33]:

nlp = spacy.load("./model-best")
text = [
'''Qualifications
- A thorough understanding of C# and .NET Core
- Knowledge of good database design and usage
- An understanding of NoSQL principles
- Excellent problem solving and critical thinking skills
- Curious about new technologies
- Experience building cloud hosted, scalable web services
- Azure experience is a plus
Requirements
- Bachelor's degree in Computer Science or related field
(Equivalent experience can substitute for earned educational qualifications)
- Minimum 4 years experience with C# and .NET
- Minimum 4 years overall experience in developing commercial software
'''
]
for doc in nlp.pipe(text, disable=["tagger", "parser"]):
    print([(ent.text, ent.label_) for ent in doc.ents])

[('C#', 'SKILLS'), ('.NET', 'SKILLS'), ('database design', 'SKILLS'), ('usage\n', 'SKILLS'), ('NoSQL', 'SKILLS'), ('problem solving', 'SKILLS'), ('critical thinking skills', 'SKILLS'), ('building cloud hosted', 'SKILLS'), ('scalable web services\n- Azure', 'SKILLS'), ('Bachelor', 'DIPLOMA'), ('Computer Science', 'DIPLOMA_MAJOR'), ('4 years', 'EXPERIENCE'), ('C#', 'SKILLS'), ('.NET', 'SKILLS'), ('4 years', 'EXPERIENCE'), ('developing commercial software\n', 'SKILLS')]


In [34]:
cat config.cfg

[paths]
train = null
dev = null
vectors = "en_core_web_lg"
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
widt